email: `jocaxex742@alibrs.com`
password:`jocaxex742jocaxex742`

In [1]:
import numpy as np
import pandas as pd
import re

In [5]:
## Parsing
# Documents
with open('TIME.ALL', 'r') as f:
    text = f.read()
result = re.findall(r"\*TEXT\s+(\d{3})\s+(\d{2}/\d{2}(/|\s)\d{2})\s+PAGE\s+(\d{3})\n\n(.+?)(?=\*TEXT|$)", text, re.DOTALL)
docDB = {match[0]: {'id': match[0], 'date': match[1], 'page': int(match[3]), 'text': match[4]} for match in result}
docDF = pd.DataFrame(docDB).T

# Queries
with open('TIME.QUE', 'r') as f:
    text = f.read()
queryDB = re.findall(r'FIND\s+\d+\s+(.*)', text)

# Stopwords
with open('TIME.STP','r') as f:
    text = f.read()
swDB = re.findall(r"^[A-Z]+$", text, re.MULTILINE)

# Relevant docs
with open('TIME.REL', 'r') as f:
    text = f.read()
    lines = text.split("\n")
rdDB = {}
for line in lines:
    numbers = re.findall(r"\d+", line)
    if numbers:
        key = numbers[0]
        values = [int(n) for n in numbers[1:]]
        rdDB[key] = values

In [6]:
rdDict

{'1': [268, 288, 304, 308, 323, 326, 334],
 '2': [326, 334],
 '3': [326, 350, 364, 385],
 '4': [370, 378, 385, 409, 421],
 '5': [359, 370, 385, 397, 421],
 '6': [257, 268, 288, 304, 308, 323, 324, 326, 334],
 '7': [386, 408],
 '8': [339, 358],
 '9': [61, 155, 156, 242, 269, 315, 339, 358],
 '10': [61, 156, 242, 269, 339, 358],
 '11': [195, 198],
 '12': [61, 155, 156, 242, 269, 339, 358],
 '13': [87, 170, 185],
 '14': [269],
 '15': [94, 118, 128, 164, 424],
 '16': [169, 170, 239],
 '17': [303, 358],
 '18': [356],
 '19': [99, 100, 195, 267, 344],
 '20': [356],
 '21': [305, 318],
 '22': [41, 356],
 '23': [425],
 '24': [318],
 '25': [342],
 '26': [318, 425],
 '27': [272, 295, 306],
 '28': [189, 219, 265, 277, 360],
 '29': [192, 329],
 '30': [253, 261, 271, 293, 325],
 '31': [47, 56, 81, 103, 150, 183, 291],
 '32': [279],
 '33': [251, 335],
 '34': [294],
 '35': [318],
 '36': [217],
 '37': [403, 406],
 '38': [364],
 '39': [22, 73, 173, 189, 219, 265, 277, 360, 396],
 '40': [20, 71, 131, 148,